In [1]:
from peft import *


/home/ava/anaconda3/envs/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

In [7]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [13]:
#change peft_model_id
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     load_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
# )

peft_model_id = "/home/ava/llamaMed/results/checkpoint-10"
config = PeftConfig.from_pretrained(peft_model_id)
# model = AutoModelForCausalLM.from_pretrained(
#     # config.base_model_name_or_path,
#     "/llm/model/falcon-7b-instruct/",
#     return_dict=True,
#     quantization_config=bnb_config,
#     device_map="auto",
#     trust_remote_code=True,
# )
MODEL_NAME = "daryl149/llama-2-7b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    # load_in_8bit=True,
    device_map="auto",  # (e.g., "auto", "cpu", "cuda:1", "mps", or a GPU ordinal rank like 1)
    # torch_dtype=torch.float16,  # bfloat16
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id


model = PeftModel.from_pretrained(model, peft_model_id)


ValueError: The current `device_map` had weights offloaded to the disk. Please provide an `offload_folder` for them. Alternatively, make sure you have `safetensors` installed if the model you are using offers the weights in this format.

In [2]:
DEVICE = "cuda:0"


In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 300
generation_config_temperature = 0.3
generation_config.top_p = 0.9
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config_eod_token_id = tokenizer.eos_token_id
 

In [3]:

def generate_response(question: str) -> str:
    prompt = f"""
    {question}
    """.strip()
    encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0],skip_special_tokens=True)

    assistant_start = '<assistant>:'
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start):].strip()

In [4]:
prompt = "I am have back pain and cold. Can you suggest me some home remedie?"
print(generate_response(prompt))


NameError: name 'tokenizer' is not defined